In [3]:
!pip install pygame

  Using cached pygame-2.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 86.8 kB/s eta 0:00:0000:0100:05m


### ---------------------------------------------------------------------------------

### FINAL VERSION

In [ ]:
import numpy as np
import random
import pygame
import sys
import math

# Colors
BLUE = (0,0,255)
BLACK = (0,0,0)
RED = (255,0,0)
YELLOW = (255,255,0)
WHITE = (255,255,255)

# Board dimensions
ROW_COUNT = 6
COLUMN_COUNT = 7

# Players
PLAYER = 0
AI = 1

# Pieces
EMPTY = 0
PLAYER_PIECE = 1
AI_PIECE = 2

WINDOW_LENGTH = 4

# Initialize pygame and fonts
pygame.init()
myfont = pygame.font.SysFont("monospace", 75)
smallfont = pygame.font.SysFont("monospace", 35)

# Sizes and screen setup
display_width = COLUMN_COUNT * 100
display_height = (ROW_COUNT + 1) * 100
SQUARESIZE = 100
RADIUS = int(SQUARESIZE/2 - 5)
screen = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption("Connect4 with Twist Phase")
clock = pygame.time.Clock()

# Utility functions
def create_board():
    return np.zeros((ROW_COUNT, COLUMN_COUNT), dtype=int)

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return 0 <= col < COLUMN_COUNT and board[ROW_COUNT-1][col] == EMPTY

def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == EMPTY:
            return r
    return None

def winning_move(board, piece):
    # Horizontal, vertical, diagonals
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT):
            if all(board[r][c+i] == piece for i in range(4)): return True
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT-3):
            if all(board[r+i][c] == piece for i in range(4)): return True
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT-3):
            if all(board[r+i][c+i] == piece for i in range(4)): return True
    for c in range(COLUMN_COUNT-3):
        for r in range(3, ROW_COUNT):
            if all(board[r-i][c+i] == piece for i in range(4)): return True
    return False

def score_position(board, piece):
    # heuristic omitted for brevity; use existing score_position
    score = 0
    # center
    center = list(board[:, COLUMN_COUNT//2])
    score += center.count(piece) * 3
    # horizontal
    for r in range(ROW_COUNT):
        row = list(board[r,:])
        for c in range(COLUMN_COUNT-3):
            window = row[c:c+4]
            score += evaluate_window(window, piece)
    # vertical
    for c in range(COLUMN_COUNT):
        col = list(board[:,c])
        for r in range(ROW_COUNT-3):
            window = col[r:r+4]
            score += evaluate_window(window, piece)
    # diag
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)
    for r in range(3, ROW_COUNT):
        for c in range(COLUMN_COUNT-3):
            window = [board[r-i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)
    return score

def evaluate_window(window, piece):
    score = 0
    opp = PLAYER_PIECE if piece==AI_PIECE else AI_PIECE
    if window.count(piece)==4: score+=100
    elif window.count(piece)==3 and window.count(EMPTY)==1: score+=5
    elif window.count(piece)==2 and window.count(EMPTY)==2: score+=2
    if window.count(opp)==3 and window.count(EMPTY)==1: score-=4
    return score

def get_valid_locations(board):
    return [c for c in range(COLUMN_COUNT) if is_valid_location(board, c)]

def minimax(board, depth, alpha, beta, maximizing):
    valid = get_valid_locations(board)
    terminal = winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or not valid
    if depth==0 or terminal:
        if terminal:
            if winning_move(board, AI_PIECE): return (None, float('inf'))
            if winning_move(board, PLAYER_PIECE): return (None, -float('inf'))
            return (None, 0)
        return (None, score_position(board, AI_PIECE))
    if maximizing:
        value, col = -math.inf, random.choice(valid)
        for c in valid:
            r = get_next_open_row(board, c)
            temp = board.copy(); drop_piece(temp, r, c, AI_PIECE)
            new_score = minimax(temp, depth-1, alpha, beta, False)[1]
            if new_score>value: value, col = new_score, c
            alpha = max(alpha, value)
            if alpha>=beta: break
        return col, value
    else:
        value, col = math.inf, random.choice(valid)
        for c in valid:
            r = get_next_open_row(board, c)
            temp = board.copy(); drop_piece(temp, r, c, PLAYER_PIECE)
            new_score = minimax(temp, depth-1, alpha, beta, True)[1]
            if new_score<value: value, col = new_score, c
            beta = min(beta, value)
            if alpha>=beta: break
        return col, value

def twist_row(board, row, direction):
    old = list(board[row,:]); new=[0]*COLUMN_COUNT
    for i,val in enumerate(old): new[(i+direction)%COLUMN_COUNT]=val
    board[row,:]=new

def apply_gravity(board):
    for c in range(COLUMN_COUNT):
        col=[p for p in board[:,c] if p!=EMPTY]
        board[:,c]=col+[EMPTY]*(ROW_COUNT-len(col))

def draw_board(board):
    screen.fill(BLACK)
    # board background
    pygame.draw.rect(screen, BLUE, (0, SQUARESIZE, display_width, display_height-SQUARESIZE))
    # slots and pieces
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            y = display_height - (r+1)*SQUARESIZE + SQUARESIZE//2
            pygame.draw.circle(screen, BLACK, (c*SQUARESIZE+SQUARESIZE//2, y), RADIUS)
            if board[r][c]==PLAYER_PIECE:
                pygame.draw.circle(screen, RED, (c*SQUARESIZE+SQUARESIZE//2, y), RADIUS)
            elif board[r][c]==AI_PIECE:
                pygame.draw.circle(screen, YELLOW, (c*SQUARESIZE+SQUARESIZE//2, y), RADIUS)
    # top bar
    pygame.draw.rect(screen, BLACK, (0,0, display_width, SQUARESIZE))

# Draw numbered UI for twist

def draw_twist_ui():
    pygame.draw.rect(screen, BLACK, (0,0, display_width, SQUARESIZE))
    # Draw instruction text
    instr = smallfont.render("Click row number to twist (L=Right, R=Left)", True, WHITE)
    screen.blit(instr, (10, 10))
    # Draw row numbers
    for r in range(ROW_COUNT):
        x = r*SQUARESIZE + SQUARESIZE//2
        y = SQUARESIZE//2
        pygame.draw.circle(screen, YELLOW, (x, y), RADIUS)
        lbl = smallfont.render(str(r), True, BLACK)
        screen.blit(lbl, lbl.get_rect(center=(x, y)))

# AI twist choice
def ai_choose_twist(board):
    best_score = score_position(board, AI_PIECE)
    best_move  = (None, 0)

    for r in range(ROW_COUNT):
        for d in (1, -1):                        # +1 = right, -1 = left
            temp = board.copy()
            twist_row(temp, r, d)
            apply_gravity(temp)
            sc = score_position(temp, AI_PIECE)
            if sc > best_score:
                best_score = sc
                best_move  = (r, d)

    return best_move




# Game init
board=create_board()
turn=random.randint(PLAYER, AI)
game_over=False
ai_msg=""

turn = AI
first_move = turn

player_moves = 0
ai_moves = 0

move_before_player_twists = 6 if turn == PLAYER else 3
move_before_ai_twists = 3 if turn == PLAYER else 6

player_twist = False
ai_twist = False



move_count_after_twist = 0
# Main loop
while True:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit(); sys.exit()

        # Player twist phase
        if player_twist and turn == PLAYER:
            if event.type == pygame.MOUSEBUTTONDOWN:
                # Get row from mouse position (0-5)
                row = min(event.pos[0] // SQUARESIZE, ROW_COUNT-1)
                direction = 1 if event.button == 1 else -1
                
                twist_row(board, row, direction)
                apply_gravity(board)
                ai_msg = f"Player twisted row {row} {'right' if direction==1 else 'left'}"
                
                if winning_move(board, PLAYER_PIECE) and winning_move(board, AI_PIECE):
                    draw_board(board)
                    screen.blit(myfont.render("DRAW!", True, YELLOW), (40, 10))
                    pygame.display.update()
                    pygame.time.wait(2000)
                    game_over = True
                elif winning_move(board, PLAYER_PIECE):
                    draw_board(board)
                    screen.blit(myfont.render("PLAYER WINS!", True, YELLOW), (40, 10))
                    pygame.display.update()
                    pygame.time.wait(2000)
                    game_over = True
                
                elif winning_move(board, AI_PIECE):
                    draw_board(board)
                    screen.blit(myfont.render("AI WINS!", True, YELLOW), (40, 10))
                    pygame.display.update()
                    pygame.time.wait(2000)
                    game_over = True

                # Update display
                draw_board(board)
                draw_twist_ui()
                screen.blit(smallfont.render(ai_msg, True, WHITE), (40, 40))
                pygame.display.update()
                
                # Reset counters
                player_twist = False
                
                # if first_move == PLAYER:
                #     ai_moves = 0
                
                turn = AI
            continue

        # Player move
        if turn == PLAYER and event.type == pygame.MOUSEBUTTONDOWN and not ai_twist and not player_twist:
            col = event.pos[0] // SQUARESIZE
            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, PLAYER_PIECE)
                
                if winning_move(board, PLAYER_PIECE):
                    draw_board(board)
                    screen.blit(myfont.render("PLAYER WINS!", True, RED), (40, 10))
                    pygame.display.update()
                    pygame.time.wait(2000)
                    game_over = True
                else:
                    player_moves += 1
                    if first_move == PLAYER :

                        if player_moves % 6 == 0:
                            player_twist = True
                        else:
                            turn = AI
                    else:
                        if player_moves % 6 == 3:
                            print("player_moves")

                            player_twist = True
                        else:
                            turn = AI
                draw_board(board)

    # AI turn
    if turn == AI and not game_over and not ai_twist and not player_twist:
        screen.blit(smallfont.render("AI Thinking...", True, WHITE), (40, 40))
        pygame.display.update()
        pygame.time.wait(1000)
        col, _ = minimax(board, 5, -math.inf, math.inf, True)
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, AI_PIECE)
            
            if winning_move(board, AI_PIECE):
                draw_board(board)
                screen.blit(myfont.render("AI WINS!", True, YELLOW), (40, 10))
                pygame.display.update()
                pygame.time.wait(2000)
                game_over = True
            else:
                ai_moves += 1
                if first_move == AI :
                    if ai_moves % 6 == 0:
                        print("ai_moves")

                        ai_twist = True
                    else:
                        turn = PLAYER
                else:
                    if ai_moves % 6 == 3:
                        ai_twist = True
                    else:
                        turn = PLAYER
                
            draw_board(board)


    # AI twist phase
    if ai_twist and turn == AI and not game_over:
        screen.blit(smallfont.render("AI Thinking row to twist...", True, WHITE), (40, 40))
        pygame.display.update()
        pygame.time.wait(1000)
        r, d = ai_choose_twist(board)
        if r is not None:
            twist_row(board, r, d)
            apply_gravity(board)
            ai_msg = f"AI twisted row {r} {'right' if d==1 else 'left'}"
            
            if winning_move(board, PLAYER_PIECE) and winning_move(board, AI_PIECE):
                    draw_board(board)
                    screen.blit(myfont.render("DRAW!", True, YELLOW), (40, 10))
                    pygame.display.update()
                    pygame.time.wait(2000)
                    game_over = True

            elif winning_move(board, PLAYER_PIECE):
                    draw_board(board)
                    screen.blit(myfont.render("PLAYER WINS!", True, YELLOW), (40, 10))
                    pygame.display.update()
                    pygame.time.wait(2000)
                    game_over = True
                
            elif winning_move(board, AI_PIECE):
                draw_board(board)
                screen.blit(myfont.render("AI WINS!", True, YELLOW), (40, 10))
                pygame.display.update()
                pygame.time.wait(2000)
                game_over = True


            draw_board(board)
            screen.blit(smallfont.render(ai_msg, True, WHITE), (40, 40))
            pygame.display.update()
            ai_twist = False  
            # if first_move == AI:
            #     player_moves = 0

        turn = PLAYER 


    # Draw UI elements
    draw_board(board)
    if player_twist and turn == PLAYER:
        draw_twist_ui()
    elif turn == PLAYER and not player_twist:
        posx = pygame.mouse.get_pos()[0]
        pygame.draw.circle(screen, RED, (posx, SQUARESIZE//2), RADIUS)
    
    if ai_msg:
        screen.blit(smallfont.render(ai_msg, True, WHITE), (40, 40))
        pygame.display.update()
        pygame.time.wait(1000)
        ai_msg = ""
        screen.blit(smallfont.render(ai_msg, True, WHITE), (40, 40))


    
    pygame.display.update()
    clock.tick(60)
    
    if game_over:
        break



pygame.quit()

player_moves
ai_moves
